In [ ]:
import cats.instances.function._
import cats.syntax.functor._

In [ ]:
implicit def function1Functor[M] = new Functor[ ({ type l[k] = Function1[M, k]})#l1 ] {
    def map[A, B](fa: M => A)(f: A => B) =
        fa andThen f
}

log: String = ""

In [7]:
var log: String = ""

def add1(n: Int) = {
    log += s"add 1 to $n;\n"
    n + 1
}
def mult5(n: Int): Double = {
    log += s"mult 3 to $n;\n"
    n * 5.0
}

log: String = ""
defined function add1
defined function mult5

In [11]:
def add1(n: Int): (Int, String) = {
    (n + 1, s"add 1 to $n;\n")
}
def mult5(n: Int): (Double, String) = {
    (n * 5.0, s"mult 3 to $n;\n")
}

defined function add1
defined function mult5

In [14]:
// string도 제너릭 하게 하려면 semigroup으로
def compose[A,B](f: A => (A, String), g: A => (B, String)):
    A => (B, String) = {
        a: A => 
        val (r1, l1) = f(a)
        val (r2, l2) = g(r1)
        (r2, l1 + l2)
    }

// kleisli arrow(compose) or fish operator
def kompose[A,B,C,F[_]](f: A => F[B], g: B => F[C]): A => F[C] = {
    a: A =>
    flatMap(f(a))(g): F[C]
}

// flatMap :: F[A] => (A => F[B]) => F[B]
def flatMap[A,B,F[_]](fa: F[A])(f: A => F[B]): F[B] = ???

defined function compose
defined function kompose
defined function flatMap

In [36]:
import scala.language.higherKinds

trait Functor[F[_]] {
    def map[A,B](fa: F[A])(f: A => B): F[B]
}
// Monad type Class
/*
    Monad primitive combinators
    1. flatMap + pure
    2. join + map + pure
    3. compose + pure
*/
trait Monad[F[_]] extends Functor[F]{
    // primitive operations
    def flatMap[A,B](fa: F[A])(f: A => F[B]): F[B]
    def pure[A](a: A): F[A]
    
    def join[A](ffa: F[F[A]]): F[A] // flatten
    
    
    // derived operations
    override def map[A,B](fa: F[A])(f: A => B): F[B] =
        flatMap(fa)(a => pure(f(a)))
    
    def map2[A,B,Z](fa: F[A], fb: F[B])(f: (A, B) => Z): F[Z] =
        flatMap(fa)(a => map(fb)(b => f(a,b)))
    
    
    def product_[A,B](fa: F[A], fb: F[B]): F[(A,B)] =
        map2(fa, fb)((_, _))
    
    def product[A,B](fa: F[A], fb: F[B]): F[(A,B)] =
        flatMap(fa)(a => map(fb)(b => (a,b)))
    
    def traverse[A,B](as: List[A])(f: A => F[B]): F[List[B]] = 
        as.foldRight(pure(List.empty[B])){ (a, acc) => 
            map2(f(a), acc)(_ :: _)
        }
    
    def sequence[A](as: List[F[A]]): F[List[A]] = 
        traverse(as)(identity)
    
    def filter[A](as: List[A])(f: A => Boolean): List[A] = ??? //foldRight 사용해보기

    def filterM[A](as: List[A])(f: A => F[Boolean]): F[List[A]] = ???
}

object Monad {
    def apply[F[_]: Monad] = implicitly[Monad[F]]
    
    implicit val optionMonad: Monad[Option] = new Monad[Option] {
        override def flatMap[A,B](fa: Option[A])(f: A => Option[B]): Option[B] = fa match {
            case None => None
            case Some(v) => f(v)
        } // Option에는 이미 flatMap이 있으니, fa.flatMap(f) 로 해도됨
        override def pure[A](a: A): Option[A] = Some(a)
    }
    
    implicit val listMonad: Monad[List] = new Monad[List] {
        def flatMap[A,B](fa: List[A])(f: A => List[B]): List[B] = fa match {
            case Nil => Nil
            case h :: t => f(h) ++ flatMap(t)(f)
        } // Option에는 이미 flatMap이 있으니, fa.flatMap(f) 로 해도됨
        def pure[A](a: A): List[A] = List(a)
    }
}

/** Monad Laws
    1. left identity
        compose(pure, f) == f
        F.pure(a).flatMap(f) == f(a)  // a: A
        
    2. right identity
        compose(f, pure) == f
        F.flatMap(fa)(F.pure) = fa   // fa: F[A]
    
    3. associativity
        compose(compose(f, g), h) == compose(f, compose(g, h))
        m.flatMap(f).flatMap(g) == m.flatMap(x => f(x).glatMap(g)) // m: F[A] 
        
*/

cmd36.sc:45: object creation impossible, since method join in trait Monad of type [A](ffa: Option[Option[A]])Option[A] is not defined
    implicit val optionMonad: Monad[Option] = new Monad[Option] {
                                                  ^cmd36.sc:53: object creation impossible, since method join in trait Monad of type [A](ffa: List[List[A]])List[A] is not defined
    implicit val listMonad: Monad[List] = new Monad[List] {
                                              ^

: 

In [20]:
Monad[List]

defined object Monad

In [34]:
// traverse
Monad[Option].traverse(List(1,2,3,4))(a => Option(a * a))

Monad[List].traverse(List(1,2))(a => List(a, a*a, a*2))

res33_0: Option[List[Int]] = Some(List(1, 4, 9, 16))
res33_1: List[List[Int]] = List(
  List(1, 2),
  List(1, 4),
  List(1, 4),
  List(1, 2),
  List(1, 4),
  List(1, 4),
  List(2, 2),
  List(2, 4),
  List(2, 4)
)

In [8]:
add1(10)

res7: Int = 11

In [9]:
mult5(3)

res8: Double = 15.0

In [10]:
log

res9: String = """
add 1 to 10;
mult 3 to 3;

"""